In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler

import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model

def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--lr_backbone', default=1e-5, type=float)
    parser.add_argument('--batch_size', default=2, type=int)
    parser.add_argument('--weight_decay', default=1e-4, type=float)
    parser.add_argument('--epochs', default=300, type=int)
    parser.add_argument('--lr_drop', default=200, type=int)
    parser.add_argument('--clip_max_norm', default=0.1, type=float,
                        help='gradient clipping max norm')

    # Model parameters
    parser.add_argument('--frozen_weights', type=str, default=None,
                        help="Path to the pretrained model. If set, only the mask head will be trained")
    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
    parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
    parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")

    # * Transformer
    parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
    parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
    parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
    parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
    parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer")
    parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
    parser.add_argument('--num_queries', default=100, type=int,
                        help="Number of query slots")
    parser.add_argument('--pre_norm', action='store_true')

    # * Segmentation
    parser.add_argument('--masks', action='store_true',
                        help="Train segmentation head if the flag is provided")

    # Loss
    parser.add_argument('--no_aux_loss', dest='aux_loss', action='store_false',
                        help="Disables auxiliary decoding losses (loss at each layer)")
    # * Matcher
    parser.add_argument('--set_cost_class', default=1, type=float,
                        help="Class coefficient in the matching cost")
    parser.add_argument('--set_cost_bbox', default=5, type=float,
                        help="L1 box coefficient in the matching cost")
    parser.add_argument('--set_cost_giou', default=2, type=float,
                        help="giou box coefficient in the matching cost")
    # * Loss coefficients
    parser.add_argument('--mask_loss_coef', default=1, type=float)
    parser.add_argument('--dice_loss_coef', default=1, type=float)
    parser.add_argument('--bbox_loss_coef', default=5, type=float)
    parser.add_argument('--giou_loss_coef', default=2, type=float)
    parser.add_argument('--eos_coef', default=0.1, type=float,
                        help="Relative classification weight of the no-object class")

    # dataset parameters
    parser.add_argument('--dataset_file', default='coco')
    parser.add_argument('--coco_path', type=str)
    parser.add_argument('--coco_panoptic_path', type=str)
    parser.add_argument('--remove_difficult', action='store_true')

    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--resume', default='', help='resume from checkpoint')
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--num_workers', default=2, type=int)

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    return parser


def main(args):
    utils.init_distributed_mode(args)
    print("git:\n  {}\n".format(utils.get_sha()))

    if args.frozen_weights is not None:
        assert args.masks, "Frozen training is meant for segmentation only"
    print(args)

    device = torch.device(args.device)

    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    model, criterion, postprocessors = build_model(args)
    model.to(device)

    model_without_ddp = model
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        model_without_ddp = model.module
    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('number of params:', n_parameters)

    param_dicts = [
        {"params": [p for n, p in model_without_ddp.named_parameters() if "backbone" not in n and p.requires_grad]},
        {
            "params": [p for n, p in model_without_ddp.named_parameters() if "backbone" in n and p.requires_grad],
            "lr": args.lr_backbone,
        },
    ]
    optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
                                  weight_decay=args.weight_decay)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

    dataset_train = build_dataset(image_set='train', args=args)
    dataset_val = build_dataset(image_set='val', args=args)

    if args.distributed:
        sampler_train = DistributedSampler(dataset_train)
        sampler_val = DistributedSampler(dataset_val, shuffle=False)
    else:
        sampler_train = torch.utils.data.RandomSampler(dataset_train)
        sampler_val = torch.utils.data.SequentialSampler(dataset_val)

    batch_sampler_train = torch.utils.data.BatchSampler(
        sampler_train, args.batch_size, drop_last=True)

    data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train,
                                   collate_fn=utils.collate_fn, num_workers=args.num_workers)
    data_loader_val = DataLoader(dataset_val, args.batch_size, sampler=sampler_val,
                                 drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)

    if args.dataset_file == "coco_panoptic":
        # We also evaluate AP during panoptic training, on original coco DS
        coco_val = datasets.coco.build("val", args)
        base_ds = get_coco_api_from_dataset(coco_val)
    else:
        base_ds = get_coco_api_from_dataset(dataset_val)

    if args.frozen_weights is not None:
        checkpoint = torch.load(args.frozen_weights, map_location='cpu')
        model_without_ddp.detr.load_state_dict(checkpoint['model'])

    output_dir = Path(args.output_dir)
    if args.resume:
        if args.resume.startswith('https'):
            checkpoint = torch.hub.load_state_dict_from_url(
                args.resume, map_location='cpu', check_hash=True)
        else:
            checkpoint = torch.load(args.resume, map_location='cpu')
        model_without_ddp.load_state_dict(checkpoint['model'])
        if not args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
            lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
            args.start_epoch = checkpoint['epoch'] + 1

    if args.eval:
        test_stats, coco_evaluator = evaluate(model, criterion, postprocessors,
                                              data_loader_val, base_ds, device, args.output_dir)
        if args.output_dir:
            utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")
        return

    print("Start training")
    start_time = time.time()
    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            sampler_train.set_epoch(epoch)
        train_stats = train_one_epoch(
            model, criterion, data_loader_train, optimizer, device, epoch,
            args.clip_max_norm)
        lr_scheduler.step()
        if args.output_dir:
            checkpoint_paths = [output_dir / 'checkpoint.pth']
            # extra checkpoint before LR drop and every 100 epochs
            if (epoch + 1) % args.lr_drop == 0 or (epoch + 1) % 100 == 0:
                checkpoint_paths.append(output_dir / f'checkpoint{epoch:04}.pth')
            for checkpoint_path in checkpoint_paths:
                utils.save_on_master({
                    'model': model_without_ddp.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'lr_scheduler': lr_scheduler.state_dict(),
                    'epoch': epoch,
                    'args': args,
                }, checkpoint_path)

        test_stats, coco_evaluator = evaluate(
            model, criterion, postprocessors, data_loader_val, base_ds, device, args.output_dir
        )

        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     **{f'test_{k}': v for k, v in test_stats.items()},
                     'epoch': epoch,
                     'n_parameters': n_parameters}

        if args.output_dir and utils.is_main_process():
            with (output_dir / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")

            # for evaluation logs
            if coco_evaluator is not None:
                (output_dir / 'eval').mkdir(exist_ok=True)
                if "bbox" in coco_evaluator.coco_eval:
                    filenames = ['latest.pth']
                    if epoch % 50 == 0:
                        filenames.append(f'{epoch:03}.pth')
                    for name in filenames:
                        torch.save(coco_evaluator.coco_eval["bbox"].eval,
                                   output_dir / "eval" / name)

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

In [2]:
parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])
args = parser.parse_args(['--batch_size=1', '--coco_path=data/', '--output_dir=data/SwinDetr', '--epochs=100', '--num_workers=4', '--resume=data/SwinDetr/checkpoint.pth'])
if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
main(args)

Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=1, weight_decay=0.0001, epochs=100, lr_drop=200, clip_max_norm=0.1, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, masks=False, aux_loss=True, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.1, dataset_file='coco', coco_path='data/', coco_panoptic_path=None, remove_difficult=False, output_dir='data/SwinDetr', device='cuda', seed=42, resume='data/SwinDetr/checkpoint.pth', start_epoch=0, eval=False, num_workers=4, world_size=1, dist_url='env://', distributed=False)
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.018182
Block Initial Type: W, drop_path_rate:0.036364
Block Initial Type: 

fatal: not a git repository (or any of the parent directories): .git


number of params: 45274842
loading annotations into memory...
Done (t=8.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
Start training
Epoch: [1]  [     0/118287]  eta: 1 day, 6:37:39  lr: 0.000100  class_error: 100.00  loss: 6.2275 (6.2275)  loss_ce: 2.9303 (2.9303)  loss_bbox: 1.4406 (1.4406)  loss_giou: 1.8566 (1.8566)  loss_ce_unscaled: 2.9303 (2.9303)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2881 (0.2881)  loss_giou_unscaled: 0.9283 (0.9283)  cardinality_error_unscaled: 8.0000 (8.0000)  time: 0.9321  data: 0.3551  max mem: 2441
Epoch: [1]  [  1000/118287]  eta: 3:47:02  lr: 0.000100  class_error: 100.00  loss: 4.3847 (4.5931)  loss_ce: 1.3935 (1.8165)  loss_bbox: 1.3104 (1.3377)  loss_giou: 1.4032 (1.4389)  loss_ce_unscaled: 1.3935 (1.8165)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2621 (0.2675)  loss_giou_unscaled: 0.7016 (0.7195)  cardinality_error_unscaled: 

Epoch: [1]  [ 19000/118287]  eta: 3:11:15  lr: 0.000100  class_error: 100.00  loss: 3.9941 (4.5615)  loss_ce: 1.6048 (1.8350)  loss_bbox: 1.1276 (1.3282)  loss_giou: 1.2616 (1.3983)  loss_ce_unscaled: 1.6048 (1.8350)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2255 (0.2656)  loss_giou_unscaled: 0.6308 (0.6991)  cardinality_error_unscaled: 4.0000 (6.5441)  time: 0.1172  data: 0.0028  max mem: 3677
Epoch: [1]  [ 20000/118287]  eta: 3:09:22  lr: 0.000100  class_error: 100.00  loss: 4.7898 (4.5629)  loss_ce: 1.8554 (1.8364)  loss_bbox: 1.2803 (1.3281)  loss_giou: 1.5184 (1.3983)  loss_ce_unscaled: 1.8554 (1.8364)  class_error_unscaled: 100.0000 (99.9994)  loss_bbox_unscaled: 0.2561 (0.2656)  loss_giou_unscaled: 0.7592 (0.6992)  cardinality_error_unscaled: 6.0000 (6.5619)  time: 0.1137  data: 0.0027  max mem: 3677
Epoch: [1]  [ 21000/118287]  eta: 3:07:30  lr: 0.000100  class_error: 100.00  loss: 4.7683 (4.5643)  loss_ce: 2.2589 (1.8371)  loss_bbox: 1.2705 (1.3284)  lo

Epoch: [1]  [ 39000/118287]  eta: 2:32:58  lr: 0.000100  class_error: 100.00  loss: 4.4130 (4.5537)  loss_ce: 1.8481 (1.8372)  loss_bbox: 1.1124 (1.3210)  loss_giou: 1.3661 (1.3955)  loss_ce_unscaled: 1.8481 (1.8372)  class_error_unscaled: 100.0000 (99.9997)  loss_bbox_unscaled: 0.2225 (0.2642)  loss_giou_unscaled: 0.6831 (0.6978)  cardinality_error_unscaled: 4.0000 (6.5575)  time: 0.1278  data: 0.0029  max mem: 3677
Epoch: [1]  [ 40000/118287]  eta: 2:31:03  lr: 0.000100  class_error: 100.00  loss: 4.4097 (4.5537)  loss_ce: 1.8629 (1.8375)  loss_bbox: 1.2106 (1.3209)  loss_giou: 1.5096 (1.3954)  loss_ce_unscaled: 1.8629 (1.8375)  class_error_unscaled: 100.0000 (99.9997)  loss_bbox_unscaled: 0.2421 (0.2642)  loss_giou_unscaled: 0.7548 (0.6977)  cardinality_error_unscaled: 4.0000 (6.5639)  time: 0.1177  data: 0.0028  max mem: 3677
Epoch: [1]  [ 41000/118287]  eta: 2:29:08  lr: 0.000100  class_error: 100.00  loss: 4.4607 (4.5533)  loss_ce: 1.6913 (1.8373)  loss_bbox: 1.4383 (1.3208)  los

...

Epoch: [12]  [ 31000/118287]  eta: 2:53:15  lr: 0.000100  class_error: 100.00  loss: 4.3230 (4.4980)  loss_ce: 1.3803 (1.8283)  loss_bbox: 1.3851 (1.2885)  loss_giou: 1.2653 (1.3812)  loss_ce_unscaled: 1.3803 (1.8283)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2770 (0.2577)  loss_giou_unscaled: 0.6327 (0.6906)  cardinality_error_unscaled: 2.0000 (6.6387)  time: 0.1232  data: 0.0030  max mem: 3677
Epoch: [12]  [ 32000/118287]  eta: 2:51:14  lr: 0.000100  class_error: 100.00  loss: 4.2716 (4.4975)  loss_ce: 1.3654 (1.8287)  loss_bbox: 1.3840 (1.2881)  loss_giou: 1.2374 (1.3806)  loss_ce_unscaled: 1.3654 (1.8287)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2768 (0.2576)  loss_giou_unscaled: 0.6187 (0.6903)  cardinality_error_unscaled: 3.0000 (6.6470)  time: 0.1098  data: 0.0028  max mem: 3677
Epoch: [12]  [ 33000/118287]  eta: 2:49:18  lr: 0.000100  class_error: 100.00  loss: 4.5957 (4.4947)  loss_ce: 1.8508 (1.8272)  loss_bbox: 1.4518 (1.2879)

Epoch: [12]  [ 51000/118287]  eta: 2:13:38  lr: 0.000100  class_error: 100.00  loss: 4.4137 (4.4884)  loss_ce: 1.7536 (1.8248)  loss_bbox: 1.2337 (1.2880)  loss_giou: 1.5382 (1.3756)  loss_ce_unscaled: 1.7536 (1.8248)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2467 (0.2576)  loss_giou_unscaled: 0.7691 (0.6878)  cardinality_error_unscaled: 4.0000 (6.6196)  time: 0.1144  data: 0.0029  max mem: 3677
Epoch: [12]  [ 52000/118287]  eta: 2:11:38  lr: 0.000100  class_error: 100.00  loss: 3.8575 (4.4878)  loss_ce: 1.4603 (1.8249)  loss_bbox: 1.1585 (1.2876)  loss_giou: 1.2031 (1.3753)  loss_ce_unscaled: 1.4603 (1.8249)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2317 (0.2575)  loss_giou_unscaled: 0.6015 (0.6876)  cardinality_error_unscaled: 3.0000 (6.6193)  time: 0.1104  data: 0.0029  max mem: 3677
Epoch: [12]  [ 53000/118287]  eta: 2:09:39  lr: 0.000100  class_error: 100.00  loss: 5.0529 (4.4875)  loss_ce: 1.7456 (1.8241)  loss_bbox: 1.3515 (1.2877)

Epoch: [12]  [ 71000/118287]  eta: 1:33:52  lr: 0.000100  class_error: 100.00  loss: 4.1972 (4.4828)  loss_ce: 1.7235 (1.8206)  loss_bbox: 1.3272 (1.2880)  loss_giou: 1.3027 (1.3741)  loss_ce_unscaled: 1.7235 (1.8206)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2654 (0.2576)  loss_giou_unscaled: 0.6514 (0.6871)  cardinality_error_unscaled: 5.0000 (6.5831)  time: 0.1122  data: 0.0027  max mem: 3677
Epoch: [12]  [ 72000/118287]  eta: 1:31:53  lr: 0.000100  class_error: 100.00  loss: 4.4370 (4.4833)  loss_ce: 1.5037 (1.8210)  loss_bbox: 1.2459 (1.2880)  loss_giou: 1.2312 (1.3743)  loss_ce_unscaled: 1.5037 (1.8210)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2492 (0.2576)  loss_giou_unscaled: 0.6156 (0.6871)  cardinality_error_unscaled: 4.0000 (6.5833)  time: 0.1222  data: 0.0029  max mem: 3677
Epoch: [12]  [ 73000/118287]  eta: 1:29:53  lr: 0.000100  class_error: 100.00  loss: 4.1624 (4.4824)  loss_ce: 1.8333 (1.8204)  loss_bbox: 1.0470 (1.2881)

Epoch: [12]  [ 91000/118287]  eta: 0:54:10  lr: 0.000100  class_error: 100.00  loss: 4.3782 (4.4836)  loss_ce: 1.7258 (1.8219)  loss_bbox: 1.2361 (1.2883)  loss_giou: 1.3396 (1.3734)  loss_ce_unscaled: 1.7258 (1.8219)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2472 (0.2577)  loss_giou_unscaled: 0.6698 (0.6867)  cardinality_error_unscaled: 4.0000 (6.5841)  time: 0.1180  data: 0.0030  max mem: 3677
Epoch: [12]  [ 92000/118287]  eta: 0:52:11  lr: 0.000100  class_error: 100.00  loss: 4.5058 (4.4837)  loss_ce: 1.8027 (1.8219)  loss_bbox: 1.3017 (1.2885)  loss_giou: 1.3971 (1.3734)  loss_ce_unscaled: 1.8027 (1.8219)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2603 (0.2577)  loss_giou_unscaled: 0.6986 (0.6867)  cardinality_error_unscaled: 5.0000 (6.5839)  time: 0.1065  data: 0.0027  max mem: 3677
Epoch: [12]  [ 93000/118287]  eta: 0:50:12  lr: 0.000100  class_error: 100.00  loss: 4.0641 (4.4835)  loss_ce: 1.5184 (1.8218)  loss_bbox: 1.1421 (1.2884)

Epoch: [12]  [111000/118287]  eta: 0:14:27  lr: 0.000100  class_error: 100.00  loss: 3.6846 (4.4826)  loss_ce: 1.3795 (1.8222)  loss_bbox: 1.2961 (1.2874)  loss_giou: 1.2107 (1.3730)  loss_ce_unscaled: 1.3795 (1.8222)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2592 (0.2575)  loss_giou_unscaled: 0.6054 (0.6865)  cardinality_error_unscaled: 2.0000 (6.5862)  time: 0.1245  data: 0.0030  max mem: 3677
Epoch: [12]  [112000/118287]  eta: 0:12:28  lr: 0.000100  class_error: 100.00  loss: 4.4750 (4.4834)  loss_ce: 2.0056 (1.8228)  loss_bbox: 1.1930 (1.2874)  loss_giou: 1.4128 (1.3731)  loss_ce_unscaled: 2.0056 (1.8228)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.2386 (0.2575)  loss_giou_unscaled: 0.7064 (0.6866)  cardinality_error_unscaled: 4.0000 (6.5897)  time: 0.1137  data: 0.0029  max mem: 3677
Epoch: [12]  [113000/118287]  eta: 0:10:29  lr: 0.000100  class_error: 100.00  loss: 4.0422 (4.4825)  loss_ce: 1.5388 (1.8223)  loss_bbox: 1.3895 (1.2874)